In [10]:
import json
import os
import pandas as pd
from PyPDF2 import PdfFileReader

pd.set_option('max_colwidth', 100)
from pandas.io.json import json_normalize
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
path = 'F://Environmental Baseline Data//Version 2//Data//text_ESA.json'

In [11]:
#read json file that contains all text
df = pd.read_json('F:/Environmental Baseline Data/Version 2/Data/text_ESA.json', lines=True)
df = df.transpose().reset_index()
df.columns = ['file_name', 'text']
df.head()

,file_name,text
0,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...
1,A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 1.txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...
2,A0H8C2 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 2 .txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...
3,A0H8C3 %2D 13.0 EIA %2D Section 13.8 to 13.13.txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...
4,A63951%2D1 V3 %2D ESA_Sec_1%2D11 %2D A4D9H6.txt,Environmental and Socio-economic AssessmentOctober 2014EASTERN MAINLINE PROJECTEnvironmental and...


In [15]:
df['text_lower'] = df.text.str.lower() #convert string to lowercase
df['legend_count'] = df['text_lower'].str.count('legend') #count instance of string 'legend'
df.head()

,file_name,text,text_lower,legend_count
0,A0H8C0 %2D 13.0 EIA %2D Section 13.1 to 13.6 .txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...,microsoft word - cp1045 master document-march 10 ak.doc environmental impact assessmentand mitig...,1
1,A0H8C1 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 1.txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...,microsoft word - cp1045 master document-march 10 ak.doc environmental impact assessmentand mitig...,0
2,A0H8C2 %2D 13.0 EIA %2D Section 13.7 Wildlife Part 2 .txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...,microsoft word - cp1045 master document-march 10 ak.doc environmental impact assessmentand mitig...,0
3,A0H8C3 %2D 13.0 EIA %2D Section 13.8 to 13.13.txt,Microsoft Word - CP1045 MASTER DOCUMENT-MARCH 10 ak.doc ENVIRONMENTAL IMPACT ASSESSMENTAND MITIG...,microsoft word - cp1045 master document-march 10 ak.doc environmental impact assessmentand mitig...,1
4,A63951%2D1 V3 %2D ESA_Sec_1%2D11 %2D A4D9H6.txt,Environmental and Socio-economic AssessmentOctober 2014EASTERN MAINLINE PROJECTEnvironmental and...,environmental and socio-economic assessmentoctober 2014eastern mainline projectenvironmental and...,10


In [92]:
# df with instance of 'legend' in text

df_legends = df[df['legend_count'] != 0] #occurance of 'legend' greater than one
df_legends.shape

postal_series = df_legends['file_name'].str.extract('([a-zA-Z]\d[a-zA-Z]\d[a-zA-Z]\d)')
df_legends.insert(1, 'Filing ID', postal_series)

df_legends = df_legends.drop(['file_name', 'text', 'text_lower'], axis = 1)

(319, 4)

In [93]:
#open Index 2 for page count
index_2 = pd.read_csv('F:/Environmental Baseline Data/Version 3/Indices/Final/Index 2 - List of PDFs for Major Projects with ESAs.csv',
                     usecols = ['Filing ID', 'Page Count'])

df_legends = df_legends.merge(index_2, on = 'Filing ID')
df_legends['pct pages legend'] = df_legends['legend_count'] / df_legends['Page Count'] * 100
df_legends = df_legends.sort_values(by = 'pct pages legend', ascending = False)
df_legends = df_legends[df_legends['pct pages legend'] > 20]
df_legends.shape
df_legends.head()


(168, 4)

,Filing ID,legend_count,Page Count,pct pages legend
202,A0Z1A5,24,6,400.000000
204,A0Z1A7,24,6,400.000000
201,A0Z1A4,24,6,400.000000
205,A0Z1A8,23,6,383.333333
185,A0Z1C2,23,6,383.333333


In [84]:
df_legends.shape

(141, 4)

In [94]:
df_legends.tail(100)

,Filing ID,legend_count,Page Count,pct pages legend
295,A4E6X6,5,5,100.000000
12,A4D9J4,15,15,100.000000
11,A4D9J3,36,36,100.000000
10,A4D9J2,15,15,100.000000
9,A4D9J1,6,6,100.000000
8,A4D9J0,26,26,100.000000
7,A4D9I9,15,15,100.000000
6,A4D9I8,15,15,100.000000
5,A4D9I7,15,15,100.000000
4,A4D9I6,15,15,100.000000
